## Seldon V2 Non Kubernetes Local Examples


### SKLearn Model

We use a simple sklearn iris classification model

In [1]:
!cat ./models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


Load the model

In [2]:
!seldon model load -f ./models/sklearn-iris-gs.yaml

{}


Wait for the model to be ready

In [3]:
!seldon model status iris -w ModelAvailable | jq -M .

{}


Do a REST inference call

In [7]:
!seldon model infer iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "32113a4d-d7e7-410c-82b3-921d455bb50c",
	"parameters": null,
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1
			],
			"datatype": "INT64",
			"parameters": null,
			"data": [
				2
			]
		}
	]
}


Do a gRPC inference call

In [8]:
!seldon model infer iris --inference-mode grpc \
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' | jq -M .

{
  "modelName": "iris_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


Unload the model

In [9]:
!seldon model unload iris

{}


### Tensorflow Model

In [10]:
!cat ./models/tfsimple1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: tfsimple1
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/triton/simple"
  requirements:
  - tensorflow


Load the model.

In [11]:
!seldon model load -f ./models/tfsimple1.yaml

{}


Wait for the model to be ready.

In [12]:
!seldon model status tfsimple1 -w ModelAvailable | jq -M .

{}


Get model metadata

In [13]:
!seldon model metadata tfsimple1

{
	"name": "tfsimple1_1",
	"versions": [
		"1"
	],
	"platform": "tensorflow_graphdef",
	"inputs": [
		{
			"name": "INPUT0",
			"datatype": "INT32",
			"shape": [
				-1,
				16
			]
		},
		{
			"name": "INPUT1",
			"datatype": "INT32",
			"shape": [
				-1,
				16
			]
		}
	],
	"outputs": [
		{
			"name": "OUTPUT0",
			"datatype": "INT32",
			"shape": [
				-1,
				16
			]
		},
		{
			"name": "OUTPUT1",
			"datatype": "INT32",
			"shape": [
				-1,
				16
			]
		}
	]
}


Do a REST inference call.

In [14]:
!seldon model infer tfsimple1 \
    '{"inputs":[{"name":"INPUT0","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","data":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "model_name": "tfsimple1_1",
  "model_version": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        1,
        16
      ],
      "data": [
        2,
        4,
        6,
        8,
        10,
        12,
        14,
        16,
        18,
        20,
        22,
        24,
        26,
        28,
        30,
        32
      ]
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        1,
        16
      ],
      "data": [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0
      ]
    }
  ]
}


Do a gRPC inference call

In [15]:
!seldon model infer tfsimple1 --inference-mode grpc \
    '{"model_name":"tfsimple1","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]},{"name":"INPUT1","contents":{"int_contents":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]},"datatype":"INT32","shape":[1,16]}]}' | jq -M .

{
  "modelName": "tfsimple1_1",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "OUTPUT0",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          2,
          4,
          6,
          8,
          10,
          12,
          14,
          16,
          18,
          20,
          22,
          24,
          26,
          28,
          30,
          32
        ]
      }
    },
    {
      "name": "OUTPUT1",
      "datatype": "INT32",
      "shape": [
        "1",
        "16"
      ],
      "contents": {
        "intContents": [
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0,
          0
        ]
      }
    }
  ],
  "rawOutputContents": [
    "AgAAAAQAAAAGAAAACAAAAAoAAAAMAAAADgAAABAAAAASAAAAFAAAABYAAAAYAAAAGgAAABwAAAAeAAAAIAAAAA==",
    "AAAAAAAAAAAAAA

Unload the model

In [16]:
!seldon model unload tfsimple1

{}


### Experiment

We will use two SKlearn Iris classification models to illustrate experiments.

In [17]:
!cat ./experiments/sklearn1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [18]:
!cat ./experiments/sklearn2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris2
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


Load both models.

In [19]:
!seldon model load -f ./experiments/sklearn1.yaml
!seldon model load -f ./experiments/sklearn2.yaml

{}
{}


Wait for both models to be ready.

In [20]:
!seldon model status iris | jq -M .
!seldon model status iris2 | jq -M .

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-05-26T07:14:11.599821862Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-05-26T07:14:11.599821862Z"
      },
      "modelDefn": {
        "meta": {
          "name": "iris",
          "kubernetesMeta": {
            "namespace": "seldon-mesh"
          }
        },
        "modelSpec": {
          "uri": "gs://seldon-models/mlserver/iris",
          "requirements": [
            "sklearn"
          ]
        },
        "deploymentSpec": {
          "replicas": 1,
          "minReplicas": 1
        }
      }
    }
  ]
}
{
  "modelName": "iris2",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver

Create an experiment that modifies the iris model to add a second model splitting traffic 50/50 between the two.

In [21]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
  namespace: seldon-mesh
spec:
  defaultModel: iris
  candidates:
  - modelName: iris
    weight: 50
  - modelName: iris2
    weight: 50


Start the experiment.

In [22]:
!seldon experiment start -f ./experiments/ab-default-model.yaml 

{}


Wait for the experiment to be ready.

In [23]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {
    "namespace": "seldon-mesh"
  }
}


Run a set of calls and record which route the traffic took. There should be roughly a 50/50 split.

In [24]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[iris2_1:27 iris_1:23]


Stop the experiment

In [25]:
!seldon experiment stop experiment-sample

{}


Unload both models.

In [26]:
!seldon model unload iris
!seldon model unload iris2

{}
{}
